<h4><center><center>WEB SCRAPING DE LUGARES MÁS RECOMENDADOS POR DEPARTAMENTO EN AIRBNB</center></h4>
<div align="center">
  <img src="https://brandemia.org/sites/default/files/sites/default/files/bnb_billboard_01-2000x1125.jpg" alt="Alojamiento en Airbnb" width="300"/>
</div>



In [ ]:
!pip install webdriver-manager

In [ ]:
!pip install pandas

In [ ]:
# Lista de departamentos del Perú
departamentos = [
    "Amazonas", "Áncash", "Apurímac", "Arequipa", "Ayacucho", "Cajamarca",
    "Callao", "Cusco", "Huancavelica", "Huánuco", "Ica", "Junín", "La Libertad",
    "Lambayeque", "Lima", "Loreto", "Madre de Dios", "Moquegua", "Pasco",
    "Piura", "Puno", "San Martín", "Tacna", "Tumbes", "Ucayali"
]

# Convierte el nombre del departamento a formato URL
import urllib.parse

# Lista donde se guardarán los resultados
links = []


# Recorrer cada departamento para crear su enlace correspondiente en Airbnb
for depto in departamentos:
    # Codificar el nombre del departamento para URL
    query = urllib.parse.quote(depto)
    url = f"https://www.airbnb.com.pe/s/{query}--Peru/homes" # Construir el enlace con el formato adecuado para Airbnb
    links.append({"Departamento": depto, "Link": url}) # Agregar el resultado a la lista como un diccionario

# Importar pandas para mostrar los resultados en formato de tabla
import pandas as pd
df = pd.DataFrame(links) # Crear un DataFrame con los enlaces generados
df.head(25)
    

,Departamento,Link
0,Amazonas,https://www.airbnb.com.pe/s/Amazonas--Peru/homes
1,Áncash,https://www.airbnb.com.pe/s/%C3%81ncash--Peru/...
2,Apurímac,https://www.airbnb.com.pe/s/Apur%C3%ADmac--Per...
3,Arequipa,https://www.airbnb.com.pe/s/Arequipa--Peru/homes
4,Ayacucho,https://www.airbnb.com.pe/s/Ayacucho--Peru/homes
5,Cajamarca,https://www.airbnb.com.pe/s/Cajamarca--Peru/homes
6,Callao,https://www.airbnb.com.pe/s/Callao--Peru/homes
7,Cusco,https://www.airbnb.com.pe/s/Cusco--Peru/homes
8,Huancavelica,https://www.airbnb.com.pe/s/Huancavelica--Peru...
9,Huánuco,https://www.airbnb.com.pe/s/Hu%C3%A1nuco--Peru...


In [ ]:
from time import sleep
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Configuración del navegador
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.97 Safari/537.36")
# opts.add_argument("--headless")  # Opcional: ejecuta sin abrir navegador

# Inicializar el navegador con ChromeDriver automáticamente
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Lista de departamentos del Perú
departamentos = [
    "Amazonas", "Áncash", "Apurímac", "Arequipa", "Ayacucho", "Cajamarca", "Callao",
    "Cusco", "Huancavelica", "Huánuco", "Ica", "Junín", "La Libertad", "Lambayeque",
    "Lima", "Loreto", "Madre de Dios", "Moquegua", "Pasco", "Piura", "Puno",
    "San Martín", "Tacna", "Tumbes", "Ucayali"
]

# Crear archivo CSV donde se guardarán los datos
with open("Lugares_recomendados_peru_airbnb.csv", mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(["departamento", "lugar_recomendado", "descripcion", "precio", "duracion", "url"])

    # Iterar por cada departamento
    for depto in departamentos:
        print(f"\n Procesando: {depto}")
        url = f"https://www.airbnb.com.pe/s/{depto.replace(' ', '-')}-–Peru/homes"
        driver.get(url)
        sleep(5) # Espera para que cargue la página

        # Scroll para cargar más resultados
        for _ in range(2):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(2)

        # Buscar todas las tarjetas de alojamiento en la página
        cards = driver.find_elements(By.XPATH, '//div[@itemprop="itemListElement"]')

        count = 0 # Contador de alojamientos extraídos por departamento
        for card in cards:
            try:
                # Título
                lugar = card.find_element(By.XPATH, './/div[@data-testid="listing-card-title"]').text.strip()

                # Descripción
                try:
                    descrip = card.find_element(By.XPATH, './/span[contains(@class, "t6mzqp7")]').text.strip()
                except:
                    descrip = ""

                # Precio
                try:
                    precio = card.find_element(By.XPATH, './/span[contains(@class, "umg93v9") or contains(@class, "umuerxh")]').text.strip()
                except:
                    precio = ""

                # Duración
                try:
                    duracion = card.find_element(By.XPATH, './/span[contains(@class, "q1vzye7p") or contains(@class, "q13rtw21")]').text.strip()
                except:
                    duracion = ""

                # Enlace al alojamiento
                try:
                    link = card.find_element(By.TAG_NAME, 'a').get_attribute('href')
                except:
                    link = ""

                # Guardar en CSV
                writer.writerow([depto, lugar, descrip, precio, duracion, link])
                count += 1

                # Limitar a los primeros 10 alojamientos por departamento
                if count == 10:
                    break

            except Exception as e:
                continue    # Si falla una tarjeta, continúa con la siguiente

# Cerrar navegador
driver.quit()
print("\n Scraping completo. Datos guardados en 'Lugares_recomendados_peru_airbnb.csv'")



 Procesando: Amazonas

 Procesando: Áncash

 Procesando: Apurímac

 Procesando: Arequipa

 Procesando: Ayacucho

 Procesando: Cajamarca

 Procesando: Callao

 Procesando: Cusco

 Procesando: Huancavelica

 Procesando: Huánuco

 Procesando: Ica

 Procesando: Junín

 Procesando: La Libertad

 Procesando: Lambayeque

 Procesando: Lima

 Procesando: Loreto

 Procesando: Madre de Dios

 Procesando: Moquegua

 Procesando: Pasco

 Procesando: Piura

 Procesando: Puno

 Procesando: San Martín

 Procesando: Tacna

 Procesando: Tumbes

 Procesando: Ucayali

 Scraping completo. Datos guardados en 'Lugares_recomendados_peru_airbnb.csv'


In [ ]:
df = pd.read_csv("Lugares_recomendados_peru_airbnb.csv")
df.head()

,departamento,lugar_recomendado,descripcion,precio,duracion,url
0,Amazonas,Habitación en Chachapoyas,Acogedora habitación,S/280,por 5 noches,https://www.airbnb.com.pe/rooms/11211265233403...
1,Amazonas,Apartamento en Chachapoyas,"Hermoso departamento con dos cuartos , chimenea",S/762,por 5 noches,https://www.airbnb.com.pe/rooms/61911853458388...
2,Amazonas,Alojamiento en Chachapoyas,Acogedora casa en Chachapoyas,S/504,por 5 noches,https://www.airbnb.com.pe/rooms/92232081837936...
3,Amazonas,Hotel en Chachapoyas,Tu estancia perfecta en Chachapoyas.,S/249,por 5 noches,https://www.airbnb.com.pe/rooms/14348297070736...
4,Amazonas,Habitación en Chachapoyas,Casa Z,S/268,NaN,https://www.airbnb.com.pe/rooms/14119859786345...
